checklist:
- neighborhood: clean
- subneighborhood: clean
- use_code: clean (create a data dictionary for this column)
- exception: clean
- tax_type: clean (create a data dictionary for this column)
- tax_class2: clean (create a data dictionary for this column)
- homestead: dummified and column cleaned
- assessor: cleaned (titlecase)
- building_area: empty (for now. check again once you get the full dataset)
- ward: float status. cleaning not needed
- land_area: clean (dropped commas)
- triennial_group: no cleaning necessary (float)
- owner_name: cleaned (converted to title case)
- address: cleaned (spend an hour on this!
- sale_price: cleaned (dropped commas and dollar signs)
- current_value: drop it
- new_value: drop it
- land_2017: cleaned (dropped commas and dollar signs)
- land_2018: cleaned (dropped commas and dollar signs)
- improvements_2017: cleaned (dropped commas and dollar signs)
- improvements_2018: cleaned (dropped commas and dollar signs)
- value_2017: cleaned (dropped commas and dollar signs)
- value_2018: cleaned (dropped commas and dollar signs)
- assessment_2017: cleaned (dropped commas and dollar signs)
- assessment_2018: cleaned (dropped commas and dollar signs)

I need to:

- drop duplicates (use reddit documentation)
- look at home values that are \$1
- drop non-dc values (look at city column)
- drop codes that are not residential
- df = df[df.codes.isin([11,12,16,17,13,18,24,23])].head()
- add lats and longs
- eda (sns plot)
- search for 'Not Available'

In [1]:
import numpy as np
import pandas as pd

In [2]:
path = 'otr copy 2.csv'
df = pd.read_csv(path)

In [3]:
#df = df.address.notnull() #http://bit.ly/2zk56jD

In [4]:
df.dropna(inplace=True)

In [5]:
df.shape

(47166, 29)

In [6]:
df.replace('', np.NaN, inplace = True)

In [7]:
df['neighborhood'] = df['neighborhood'].str.title()
df['neighborhood'] = df['neighborhood'].str.replace('American Univ. Park','American University Park')

In [8]:
df['use_code2'] = df['use_code'].str.extract('(^\d{2,})', expand = True)
df['use_code2']=df['use_code2'].str.strip()

In [9]:
df['tax_type2'] = df['tax_type'].str.extract('(^\w{2})', expand = True)
df.tax_type2=df['tax_type2'].str.strip()

In [10]:
df['tax_class2'] = df['tax_class'].str.extract(r'([0-9]\b)', expand = True)
df['tax_class2']=df['tax_class2'].str.strip()

In [11]:
df.homestead.value_counts()

** Not receiving the Homestead Deduction                             22223
** Currently receiving the Homestead Deduction*.                     22036
** Currently receiving the Senior Citizen Homestead Deduction*.       2752
                                                                       155
Name: homestead, dtype: int64

In [12]:
homestead1 = pd.get_dummies(df.homestead).iloc[:, 1:]
homestead1.columns

Index(['** Currently receiving the Homestead Deduction*.  ',
       '** Currently receiving the Senior Citizen Homestead Deduction*.  ',
       '** Not receiving the Homestead Deduction '],
      dtype='object')

In [13]:
homestead1.columns = ['homestead_yes', 'homestead_senior', 'homestead_no']
df = pd.concat([df, homestead1], axis=1)

In [14]:
df['assessor'] = df['assessor'].str.title()

In [15]:
df['land_area'] = df['land_area'].str.replace(',', '')

In [16]:
df['owner_name'] = df['owner_name'].str.title()

In [17]:
df['sale_price'] = df['sale_price'].str.replace(',', '').str.replace('$', '')

In [18]:
df.drop(['current_value','new_value'], axis=1, inplace=True)

In [19]:
df['land_2017']= df['land_2017'].str.replace(',', '').str.replace('$', '')

In [20]:
df['land_2018']= df['land_2018'].str.replace(',', '').str.replace('$', '')

In [21]:
df['improvements_2017']= df['improvements_2017'].str.replace(',', '').str.replace('$', '')

In [22]:
df['improvements_2018']= df['improvements_2018'].str.replace(',', '').str.replace('$', '')

In [23]:
df['value_2017']= df['value_2017'].str.replace(',', '').str.replace('$', '')

In [24]:
df['value_2018']= df['value_2018'].str.replace(',', '').str.replace('$', '')

In [25]:
df['assessment_2017']= df['assessment_2017'].str.replace(',', '').str.replace('$', '')

In [26]:
df['assessment_2018']= df['assessment_2018'].str.replace(',', '').str.replace('$', '')

In [27]:
df['address'] = df['address'].str.replace('                  ', ', ')

In [28]:
df['address'] = df['address'].str.replace('                 ', ', ')

In [29]:
df['address'] = df['address'].str.replace('    ', ' ')

In [30]:
df['address'] = df['address'].str.replace('   ', ' ')

In [31]:
df['address'] = df['address'].str.replace('  ', ' ')

In [32]:
df['address'] = df['address'].str.strip()

In [33]:
df['address1'] = df['address']

In [34]:
df['address1'] = df['address1'].str.strip()

In [35]:
df['address1'] = df['address1'].str.slice(-10, -5) #zip_code

In [36]:
df.address1.value_counts().head()

20009    8967
20016    4248
20019    4116
20008    3183
20017    2797
Name: address1, dtype: int64

In [37]:
### there's an issue above with how i slice addresses

In [38]:
df['zip_code'] = df['address1']

In [39]:
df['address1'] = df['address']

In [40]:
df['state'] = df['address1'].str.slice(-12, -10)

In [41]:
# a quick note about state, the state is derived from the address column, which is the mailing address

In [42]:
df['address1'] = df['address']

In [43]:
df['address1'] = df['address1'].str.strip()

In [44]:
df['city'] = df['address1'].str.slice(0, -14)

In [45]:
df['city'] = df['city'].str.replace('.+?(?=;)','')

In [46]:
df['city'] = df['city'].str.slice(1)  

In [47]:
df['city'] = df['city'].str.strip().str.title()

In [48]:
df['address1'] = df['address']

In [49]:
df['street'] = df['address1'].apply(lambda x: x.split(';')[0]).str.strip()

In [50]:
df.head()

,neighborhood,sub_neighborhood,use_code,exception,tax_type,tax_class,homestead,assessor,building_area,ward,...,tax_type2,tax_class2,homestead_yes,homestead_senior,homestead_no,address1,zip_code,state,city,street
0,American University Park,C,12 - Residential-Detached-Single-Fa,No,TX -Taxable,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,...,TX,1,1,0,0,"4933 MASSACHUSETTS AVE NW; WASHINGTON, DC20016...",20016,DC,Washington,4933 MASSACHUSETTS AVE NW
1,American University Park,C,12 - Residential-Detached-Single-Fa,No,TX -Taxable,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,...,TX,1,1,0,0,"4931 MASSACHUSETTS AVE NW; WASHINGTON, DC20016...",20016,DC,Washington,4931 MASSACHUSETTS AVE NW
2,American University Park,C,12 - Residential-Detached-Single-Fa,No,TX -Taxable,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,...,TX,1,1,0,0,"4923 MASSACHUSETTS AVE NW; WASHINGTON, DC20016...",20016,DC,Washington,4923 MASSACHUSETTS AVE NW
3,American University Park,C,12 - Residential-Detached-Single-Fa,No,TX -Taxable,001 - Residential,** Not receiving the Homestead Deduction,Parker Norman,,3.0,...,TX,1,0,0,1,"7200 WISCONSIN AVE STE 903; BETHESDA, MD20814-...",20814,MD,Bethesda,7200 WISCONSIN AVE STE 903
4,American University Park,C,12 - Residential-Detached-Single-Fa,No,TX -Taxable,001 - Residential,** Currently receiving the Homestead Deduction...,Parker Norman,,3.0,...,TX,1,1,0,0,"4410 49TH ST NW; WASHINGTON, DC20016-4303",20016,DC,Washington,4410 49TH ST NW


In [51]:
df.shape

(47166, 38)

In [52]:
df.duplicated(subset=['address']).sum()
#20670

df.duplicated(subset=['address', 'date']).sum()
#18304

df.duplicated(subset=['date','address']).sum()
#18304

18304

In [53]:
#dropping! the smallest number
df.drop_duplicates(subset=['address', 'date'], inplace = True)


In [54]:
df.shape

(28862, 38)

In [55]:
df.duplicated(subset=['address', 'date']).sum()
#18304

0

In [56]:
df.sort_values(['address'], ascending=False).duplicated().sum()


0

In [57]:
df.columns

Index(['neighborhood', 'sub_neighborhood', 'use_code', 'exception', 'tax_type',
       'tax_class', 'homestead', 'assessor', 'building_area', 'ward',
       'land_area', 'triennial_group', 'owner_name', 'address', 'sale_price',
       'date', 'instrument_no', 'sales_code', 'sales_type', 'land_2017',
       'land_2018', 'improvements_2017', 'improvements_2018', 'value_2017',
       'value_2018', 'assessment_2017', 'assessment_2018', 'use_code2',
       'tax_type2', 'tax_class2', 'homestead_yes', 'homestead_senior',
       'homestead_no', 'address1', 'zip_code', 'state', 'city', 'street'],
      dtype='object')

In [58]:
# ### duplicates
# df.sort_values(['address', 'date'], ascending=False).drop_duplicates(inplace = True) 
# #this drops all of the smaller duplicates based on comments count
df.neighborhood.value_counts()

Brookland                   3882
Mount Pleasant              3701
Brightwood                  3083
Kalorama                    2683
American University Park    1891
Anacostia                   1567
Ledroit Park                1537
Marshall Heights            1453
Lily Ponds                  1106
Brentwood                    874
Burleith                     646
Kent                         644
Michigan Park                610
Barry Farms                  589
Berkley                      572
Congress Heights             199
Foxhall                      199
Hawthorne                    198
Fort Lincoln                 198
Colonial Village             198
Forest Hills                 196
Eckington                    195
Capitol Hill                 194
Garfield                     191
Columbia Heights             189
Central                      189
Chillum                      187
Chevy Chase                  187
Crestwood                    187
Glover Park                  183
Foggy Bott

In [59]:
# df[df.address.duplicated()].iloc[:, 2:15].head(100)

In [60]:
df.columns

Index(['neighborhood', 'sub_neighborhood', 'use_code', 'exception', 'tax_type',
       'tax_class', 'homestead', 'assessor', 'building_area', 'ward',
       'land_area', 'triennial_group', 'owner_name', 'address', 'sale_price',
       'date', 'instrument_no', 'sales_code', 'sales_type', 'land_2017',
       'land_2018', 'improvements_2017', 'improvements_2018', 'value_2017',
       'value_2018', 'assessment_2017', 'assessment_2018', 'use_code2',
       'tax_type2', 'tax_class2', 'homestead_yes', 'homestead_senior',
       'homestead_no', 'address1', 'zip_code', 'state', 'city', 'street'],
      dtype='object')

In [61]:
df.use_code2

0        12
1        12
2        12
3        12
4        12
5        12
6        12
7        12
8        12
9        12
10       12
11       12
12       12
13       12
14       12
15       12
16       12
17       12
18       12
19       12
20       12
21       12
22       94
23       12
24       12
25       12
26       12
27       12
28       12
29       12
         ..
47065    16
47066    16
47067    16
47068    16
47069    16
47070    16
47071    16
47072    16
47073    16
47074    16
47075    16
47076    16
47077    16
47078    16
47079    23
47080    11
47081    22
47082    16
47083    16
47084    16
47085    16
47090    87
47097    49
47103    42
47113    59
47120    45
47146    48
47152    49
47165    58
47171    41
Name: use_code2, Length: 28862, dtype: object

In [62]:
# #reducing dataframe to just residential properties
# df[df.use_code2.isin([11])].head()

# come back to this

In [63]:
df.tail()

,neighborhood,sub_neighborhood,use_code,exception,tax_type,tax_class,homestead,assessor,building_area,ward,...,tax_type2,tax_class2,homestead_yes,homestead_senior,homestead_no,address1,zip_code,state,city,street
47120,Mount Pleasant,C,45 - Store-Restaurant,,TX -Taxable,,,,,1.0,...,TX,NaN,0,0,0,13609 TURNMORE RD; SILVER SPRING MD20009,NG MD,RI,Silver,13609 TURNMORE RD
47146,Mount Pleasant,C,48 - Commercial-Retail-Condo,,TX -Taxable,,,,,1.0,...,TX,NaN,0,0,0,"750 17TH ST NW SUITE 1100; WASHINGTON, DC20006","N, DC",TO,Washi,750 17TH ST NW SUITE 1100
47152,Mount Pleasant,B,49 - Commercial-Retail-Misc,,TX -Taxable,,,,,1.0,...,TX,NaN,0,0,0,"2120 L ST NW STE 315; WASHINGTON, DC20037-1527",20037,DC,Washington,2120 L ST NW STE 315
47165,Mount Pleasant,B,58 - Commercial-Office-Condo,,TX -Taxable,,,,,1.0,...,TX,NaN,0,0,0,"1729 NEWTON ST NW; WASHINGTON, DC20850","N, DC",TO,Washi,1729 NEWTON ST NW
47171,Mount Pleasant,B,41 - Store-Small 1-Story,,TX -Taxable,,,,,1.0,...,TX,NaN,0,0,0,12125 HERITAGE PARK CIR; SILVER SPRING MD20906...,20906,MD,Silver Sprin,12125 HERITAGE PARK CIR


In [64]:
df.columns

Index(['neighborhood', 'sub_neighborhood', 'use_code', 'exception', 'tax_type',
       'tax_class', 'homestead', 'assessor', 'building_area', 'ward',
       'land_area', 'triennial_group', 'owner_name', 'address', 'sale_price',
       'date', 'instrument_no', 'sales_code', 'sales_type', 'land_2017',
       'land_2018', 'improvements_2017', 'improvements_2018', 'value_2017',
       'value_2018', 'assessment_2017', 'assessment_2018', 'use_code2',
       'tax_type2', 'tax_class2', 'homestead_yes', 'homestead_senior',
       'homestead_no', 'address1', 'zip_code', 'state', 'city', 'street'],
      dtype='object')

In [65]:
df.columns

Index(['neighborhood', 'sub_neighborhood', 'use_code', 'exception', 'tax_type',
       'tax_class', 'homestead', 'assessor', 'building_area', 'ward',
       'land_area', 'triennial_group', 'owner_name', 'address', 'sale_price',
       'date', 'instrument_no', 'sales_code', 'sales_type', 'land_2017',
       'land_2018', 'improvements_2017', 'improvements_2018', 'value_2017',
       'value_2018', 'assessment_2017', 'assessment_2018', 'use_code2',
       'tax_type2', 'tax_class2', 'homestead_yes', 'homestead_senior',
       'homestead_no', 'address1', 'zip_code', 'state', 'city', 'street'],
      dtype='object')

In [66]:
import datetime as dt
import datetime
df['date']= pd.to_datetime(df.date)

In [67]:
df['qtr'] = df.date.dt.quarter
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year

In [71]:
df.to_csv('otr_clean.csv', index = False, encoding='utf-8')